In [12]:
# Step 1: Import libraries
import pandas as pd
import requests
import ast

In [14]:
# Step 2: Load and clean the data
df = pd.read_csv('recipe_subset_raw.csv')

# Standardize column names
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

# Convert stringified lists to actual lists
for col in ['ingredients', 'directions', 'ner']:
    df[col] = df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith('[') else x)

# Drop rows with missing or invalid ingredients
df = df[df['ingredients'].apply(lambda x: isinstance(x, list) and len(x) > 0)]

# Use only a subset to avoid hitting API limits
subset = df.iloc[:100].copy()

In [16]:
# Step 3: Define the cost estimation function
def estimate_recipe_cost(ingredients, servings=1, api_key="dc03366924mshc17da3ce290e748p18b0bfjsn5ca7b8892a81"):
    url = "https://spoonacular-recipe-food-nutrition-v1.p.rapidapi.com/recipes/parseIngredients"

    headers = {
        "x-rapidapi-key": api_key,
        "x-rapidapi-host": "spoonacular-recipe-food-nutrition-v1.p.rapidapi.com",
        "content-type": "application/x-www-form-urlencoded"
    }

    payload = {
        "ingredientList": "\n".join(ingredients),
        "servings": servings
    }

    response = requests.post(url, data=payload, headers=headers)

    if response.status_code != 200:
        print(f"Error {response.status_code}: {response.text}")
        return None, []

    results = response.json()

    total_cost = 0.0
    breakdown = []

    for item in results:
        name = item['name']
        cost_info = item.get('estimatedCost', {})
        cost_cents = cost_info.get('value', 0.0)
        cost_usd = cost_cents / 100.0
        total_cost += cost_usd
        breakdown.append((name, cost_usd, "USD"))

    return total_cost, breakdown

In [19]:
# Step 4: Wrapper function to apply to DataFrame
def get_price_wrapper(ingredient_list):
    try:
        total, _ = estimate_recipe_cost(ingredient_list, servings=1, api_key="dc03366924mshc17da3ce290e748p18b0bfjsn5ca7b8892a81")
        return round(total, 2)
    except:
        return None

# Apply to subset
subset['estimated_price_usd'] = subset['ingredients'].apply(get_price_wrapper)

In [20]:
# Step 5: Save the priced subset to CSV
subset.to_csv('priced_recipe_subset.csv', index=False)
print("Saved to priced_recipe_subset.csv")

Saved to priced_recipe_subset.csv
